In this project I have implemented naive bayes by my own for datas having discrete values, then trained a model on iris data as iris data have continuous values so I have first converted it into discrete values and then checked the precision and recall value. Then applied Inbuilt sklearn naive bayes classfier for continuous values create a model fit the data into it then print the classification report and compare the precision and recall value as calculated before.

In [25]:
import pandas as pd
import numpy as np
import math
from sklearn import datasets

In [26]:
iris = datasets.load_iris()
df = pd.DataFrame(iris.data)
df.columns = iris.feature_names
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


As we have implemented naive bayes for some discrete values of the features here the features are continuous so let's first 
convert it into discrete values then we will apply naive bayes on it.

In [27]:
#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label p
#if (m + Mean_Value)/2 <=val < Mean_Value then it is assigned label q
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label r
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label s


def split(val,*args):
    if(args[0] <= val < args[1]):
        return 'p'
    elif(args[1] <= val < args[2]):
        return 'q'
    elif(args[2] <= val < args[3]):
        return 'r'
    elif(args[3] <= val <= args[4]):
        return 's'
    
def to_split(df,feature_name):
    x = df[feature_name].min()
    x1 = (df[feature_name].min() + df[feature_name].mean())/2
    y =  df[feature_name].mean()
    z = (df[feature_name].max() + df[feature_name].mean())/2
    z1 = df[feature_name].max()
    return df[feature_name].apply(split,args = (x,x1,y,z,z1))

In [28]:
df['sl'] = to_split(df,iris.feature_names[0])
df['sw'] = to_split(df,iris.feature_names[1])
df['pl'] = to_split(df,iris.feature_names[2])
df['pw'] = to_split(df,iris.feature_names[3])
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),sl,sw,pl,pw
0,5.1,3.5,1.4,0.2,q,r,p,p
1,4.9,3.0,1.4,0.2,p,q,p,p
2,4.7,3.2,1.3,0.2,p,r,p,p
3,4.6,3.1,1.5,0.2,p,r,p,p
4,5.0,3.6,1.4,0.2,p,r,p,p
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,r,q,r,s
146,6.3,2.5,5.0,1.9,r,p,r,s
147,6.5,3.0,5.2,2.0,r,q,r,s
148,6.2,3.4,5.4,2.3,r,r,s,s


In [29]:
# removing the columns containing continuous data which are converted into discrete value
list = [iris.feature_names[0],iris.feature_names[1],iris.feature_names[2],iris.feature_names[3]]
df.drop(list,axis = 1,inplace = True)
df

,sl,sw,pl,pw
0,q,r,p,p
1,p,q,p,p
2,p,r,p,p
3,p,r,p,p
4,p,r,p,p
...,...,...,...,...
145,r,q,r,s
146,r,p,r,s
147,r,q,r,s
148,r,r,s,s


In [30]:
def fit(X,Y):
    result = {}
    class_values = set(Y)
    for current_class in class_values:
        result[current_class] = {}
        current_class_rows = (Y == current_class)
        X_current = X[current_class_rows]
        Y_current = Y[current_class_rows]
        result["total_data"] = len(Y)
        result[current_class]["total_count"] = len(Y_current)
        num_features = x.shape[1]
        for j in range(1, 1 + num_features):
            result[current_class][j] = {}
            all_possible_features = set(X[:,j-1])    #slight diff
            for current_value in all_possible_features:
                result[current_class][j][current_value] = (X_current[:,j-1] == current_value).sum()    #slight diff
    return result
                

In [31]:
def predict(result,X):
    Y_pred = [0] * X.shape[0]
    for i in range((X.shape[0])):
        classes = []
        keys = result.keys()
        for key in keys:
            classes.append(key)
        classes.remove("total_data")
        #classes.remove("total_count")
        p = -1
        for current_class in classes:
            x = result[current_class]['total_count']/result['total_data']
            for j in range((X.shape[1])):
                current_value = X[i][j]
                x *= (result[current_class][j+1][current_value] +1)/(result[current_class]['total_count'] + len(result[current_class][j+1].keys()))
            if(p < x):
                p = x
                Y_pred[i] = current_class
    return Y_pred
    

In [32]:
from sklearn.model_selection import train_test_split
x=df.values
y = iris.target
x_train, x_test, y_train, y_test=train_test_split(x, y)

In [33]:
x_train.shape,y_train.shape

((112, 4), (112,))

In [34]:
x_test.shape

(38, 4)

In [35]:
clf = fit(x_train,y_train)

In [36]:
y_test_pred = predict(clf,x_test)

In [37]:
li = (y_test_pred == y_test)

In [57]:
li.sum()

35

In [38]:
y_test

array([2, 0, 2, 1, 1, 0, 2, 0, 0, 1, 2, 0, 2, 1, 2, 0, 1, 0, 0, 2, 1, 1,
       1, 0, 0, 0, 2, 1, 1, 1, 2, 0, 0, 0, 1, 0, 0, 2])

In [39]:
li

array([ True,  True, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [40]:
y_train_pred = predict(clf,x_train)
li2 = (y_train_pred == y_train)
li2

array([ True, False,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [41]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_test_pred))
print(confusion_matrix(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.85      0.92      0.88        12
           2       0.89      0.80      0.84        10

    accuracy                           0.92        38
   macro avg       0.91      0.91      0.91        38
weighted avg       0.92      0.92      0.92        38

[[16  0  0]
 [ 0 11  1]
 [ 0  2  8]]


In [43]:
from sklearn.naive_bayes import GaussianNB

In [46]:
clf1 = GaussianNB()

In [50]:
x1 = iris.data
y1 = iris.target
x1_train, x1_test, y1_train, y1_test=train_test_split(x1, y1)

In [52]:
clf1.fit(x1_train, y1_train)

GaussianNB()

In [54]:
y1_test_pred = clf1.predict(x1_test)

In [58]:
li1 = (y1_test_pred == y1_test)
li1.sum()

36

In [59]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y1_test,y1_test_pred))
print(confusion_matrix(y1_test,y1_test_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.92      0.92      0.92        12
           2       0.92      0.92      0.92        12

    accuracy                           0.95        38
   macro avg       0.94      0.94      0.94        38
weighted avg       0.95      0.95      0.95        38

[[14  0  0]
 [ 0 11  1]
 [ 0  1 11]]


clearly we can see the sklearn gaussian naive bayes has better results than that calculated above and it is quite obvious also as in above implementation I have converted the continuous data into discete data whereas gaussian naive bayes worked on the 
continuous data only with the assumption that data follow gaussian distribution(Normal distribution)